In [1]:
# install octave
!sudo apt-get -qq update
!sudo apt-get -qq install octave octave-signal liboctave-dev

# install oct2py that compatible with colab
import os

from pkg_resources import get_distribution

os.system(
    f"pip install -qq"
    f" ipykernel=={get_distribution('ipykernel').version}"
    f" ipython=={get_distribution('ipython').version}"
    f" tornado=={get_distribution('tornado').version}"
    f" oct2py"
)

# install packages
!pip install -qq matpower matpowercaseframes

In [2]:
import matpower
import oct2py

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.6.1
matpower version: 7.1.0.2.1.6


In [3]:
from matpower import start_instance
from matpowercaseframes import CaseFrames
from pypglib import pglib_opf_case5_pjm

In [4]:
def runopf(mpc, m):
    m.push("_mpc", mpc, verbose=False)
    m.eval("_r1 = runopf(_mpc);", verbose=True)

    # list of avaible data, see:
    #   https://matpower.org/docs/ref/matpower7.1/lib/opf.html
    mpc["baseMVA"] = m.eval("_r1.baseMVA;", verbose=False)
    mpc["version"] = m.eval("_r1.version;", verbose=False)
    mpc["bus"] = m.eval("_r1.bus;", verbose=False)
    mpc["gen"] = m.eval("_r1.gen;", verbose=False)
    mpc["branch"] = m.eval("_r1.branch;", verbose=False)
    mpc["gencost"] = m.eval("_r1.gencost;", verbose=False)
    mpc["f"] = m.eval("_r1.f;", verbose=False)

    return mpc

In [5]:
m = start_instance()  # runpf require oct2py / matlab

In [6]:
mpc = m.loadcase(pglib_opf_case5_pjm, verbose=False)
mpc = runopf(mpc, m)


MATPOWER Version 7.1, 08-Oct-2020 -- AC Optimal Power Flow
  AC OPF formulation: polar voltages, power balance eqns
MATPOWER Interior Point Solver -- MIPS, Version 1.4, 08-Oct-2020
 (using built-in linear solver)
Converged!

Converged in 0.48 seconds
Objective Function Value = 17551.89 $/hr
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              5     Total Gen Capacity    1530.0       -1147.5 to 1147.5
Generators         5     On-line Capacity      1530.0       -1147.5 to 1147.5
Committed Gens     5     Generation (actual)   1005.2             371.7
Loads              3     Load                  1000.0             328.7
  Fixed            3       Fixed               1000.0             328.7
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Sh

In [7]:
cf = CaseFrames(mpc)

In [8]:
cf.attributes

['version', 'baseMVA', 'bus', 'gen', 'gencost', 'branch']

In [9]:
cf.branch

,F_BUS,T_BUS,BR_R,BR_X,BR_B,RATE_A,RATE_B,RATE_C,TAP,SHIFT,...,ANGMIN,ANGMAX,PF,QF,PT,QT,MU_SF,MU_ST,MU_ANGMIN,MU_ANGMAX
1,1.0,2.0,0.00281,0.0281,0.00712,400.0,400.0,400.0,0.0,0.0,...,-30.0,30.0,252.377780,-42.449742,-250.793748,57.458279,0.0,0.000000,0.0,0.0
2,1.0,4.0,0.00304,0.0304,0.00658,426.0,426.0,426.0,0.0,0.0,...,-30.0,30.0,187.868653,33.131817,-186.915290,-24.352797,0.0,0.000000,0.0,0.0
3,1.0,5.0,0.00064,0.0064,0.03126,426.0,426.0,426.0,0.0,0.0,...,-30.0,30.0,-230.246488,166.817562,230.695383,-165.930038,0.0,0.000000,0.0,0.0
4,2.0,3.0,0.00108,0.0108,0.01852,426.0,426.0,426.0,0.0,0.0,...,-30.0,30.0,-49.206252,-156.068279,49.449235,156.289420,0.0,0.000000,0.0,0.0
5,3.0,4.0,0.00297,0.0297,0.00674,426.0,426.0,426.0,0.0,0.0,...,-30.0,30.0,-24.951035,135.099963,25.417029,-131.229414,0.0,0.000000,0.0,0.0
6,4.0,5.0,0.00297,0.0297,0.00674,240.0,240.0,240.0,0.0,0.0,...,-30.0,30.0,-238.501514,13.310379,239.998344,0.891151,0.0,61.310846,0.0,0.0


In [10]:
print(f"Final objective function value: {mpc['f']} $/hr")

Final objective function value: 17551.891438196122 $/hr


In [11]:
import numpy as np


# manually calculate pf cost in python
def get_fuel_cost(cf):
    # TODO: support piecewise mode
    n_const_cost = cf.gencost["NCOST"].iloc[0]

    n_gen = len(cf.gen)
    cost = np.zeros(n_gen)

    # loop columns polynomials
    for i in range(int(n_const_cost)):
        cost += cf.gencost[f"COST_{i}"] * cf.gen["PG"] ** i

    total_cost = cost.values @ cf.gen["GEN_STATUS"].values

    return total_cost

In [12]:
fuel_cost = get_fuel_cost(cf)
print(f"Final objective function value: {fuel_cost} $/hr")

Final objective function value: 17551.891438196122 $/hr


In [16]:
assert np.isclose(fuel_cost, mpc.f)